# Clustering Crypto

In [150]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [151]:
# Use the following endpoint to fetch json data
#url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [152]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
# file_path = Path("Resources/crypto_data.csv")


In [153]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

file_path = Path("Resources/crypto_data.csv")

Crypto_df = pd.read_csv(file_path)

# Create a DataFrame
Crypto_df.head()


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [154]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
Crypto_df = Crypto_df.drop(columns=['Unnamed: 0'])
Crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [155]:
# Keep only cryptocurrencies that are trading
Crypto_df = Crypto_df.loc[Crypto_df['IsTrading'] == True]
Crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [156]:
# Keep only cryptocurrencies with a working algorithm
Crypto_df = Crypto_df.loc[Crypto_df['Algorithm'] != 'N/A']
Crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [157]:
# Remove the "IsTrading" column
Crypto_df = Crypto_df.drop(columns=['IsTrading'])
Crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0


In [158]:
# Remove rows with at least 1 null value
Crypto_df = Crypto_df.dropna()
Crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [159]:
# Remove rows with cryptocurrencies having no coins mined
Crypto_df = Crypto_df.loc[Crypto_df['TotalCoinsMined'] != 0]
Crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0


In [160]:
# Drop rows where there are 'N/A' text values
for row in Crypto_df:
    Crypto_df = Crypto_df.loc[Crypto_df[row] != 'N/A']
Crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0


In [161]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from Crypto_df
coins_name = Crypto_df[['CoinName']]
coins_name.head()

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum


In [162]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
Crypto_df = Crypto_df.drop(columns=['CoinName'])
Crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0


In [163]:
# Create dummy variables for text features
text_feature_variable = ''

# Import Label Encoder from SKLearn and encode categorical data:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(Crypto_df['Algorithm'])
Crypto_df['Algorithm'] = le.transform(Crypto_df['Algorithm'])

le.fit(Crypto_df['ProofType'])
Crypto_df['ProofType'] = le.transform(Crypto_df['ProofType'])


Crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,53,16,4.199995e+01,42
2,53,16,1.055185e+09,532000000
5,67,16,2.927942e+10,314159265359
7,47,13,1.792718e+07,21000000
8,20,13,1.076842e+08,0


In [164]:
# Standardize data
scaler = StandardScaler()

df_scaled = scaler.fit_transform(Crypto_df)
df_scaled

array([[ 0.39335561,  0.8871567 , -0.11674788, -0.15286468],
       [ 0.39335561,  0.8871567 , -0.09358885, -0.14499604],
       [ 1.21439604,  0.8871567 ,  0.52587231,  4.4937636 ],
       ...,
       [-2.12841143,  0.00878917, -0.09523411, -0.13215444],
       [-1.65924547,  0.00878917, -0.11658774, -0.15255408],
       [ 0.39335561, -1.45515672, -0.11674507, -0.15284989]])

### Reducing Dimensions Using PCA

In [165]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [166]:
# Create a DataFrame with the principal components data
crypto_pca = pca.fit_transform(df_scaled)

PCS_df = pd.DataFrame(data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=Crypto_df.index)

PCS_df.head()

,PC 1,PC 2,PC 3
0,-0.412665,0.823150,0.357227
2,-0.391306,0.827873,0.358336
5,3.143152,2.186748,0.487273
7,-0.190301,-0.003829,-0.043688
8,-0.051013,-1.161387,1.020473


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [167]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(PCS_df)
    inertia.append(km.inertia_)
# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)


:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [168]:
# Initialize the K-Means model
model = KMeans(n_clusters=6, random_state=0)

# Fit the model
model.fit(PCS_df)

# Predict clusters
predictions = model.predict(PCS_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = Crypto_df.join(PCS_df)
clustered_df['CoinName'] = coins_name['CoinName']
clustered_df['Class'] = predictions
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
0,53,16,4.199995e+01,42,-0.412665,0.823150,0.357227,42 Coin,0
2,53,16,1.055185e+09,532000000,-0.391306,0.827873,0.358336,404Coin,0
5,67,16,2.927942e+10,314159265359,3.143152,2.186748,0.487273,EliteCoin,5
7,47,13,1.792718e+07,21000000,-0.190301,-0.003829,-0.043688,Bitcoin,4
8,20,13,1.076842e+08,0,-0.051013,-1.161387,1.020473,Ethereum,1


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [169]:
# Scale data to create the scatter plot
scaled = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply","TotalCoinsMined"]])
scaled_df = pd.DataFrame(scaled, columns = ["TotalCoinSupply","TotalCoinsMined"], index = Crypto_df.index)
clustered_df = clustered_df.drop(columns=['TotalCoinsMined','TotalCoinSupply'])

clustered_df = clustered_df.join(scaled_df)
clustered_df.head()

,Algorithm,ProofType,PC 1,PC 2,PC 3,CoinName,Class,TotalCoinSupply,TotalCoinsMined
0,53,16,-0.412665,0.823150,0.357227,42 Coin,0,4.200000e-11,0.005942
2,53,16,-0.391306,0.827873,0.358336,404Coin,0,5.320000e-04,0.007002
5,67,16,3.143152,2.186748,0.487273,EliteCoin,5,3.141593e-01,0.035342
7,47,13,-0.190301,-0.003829,-0.043688,Bitcoin,4,2.100000e-05,0.005960
8,20,13,-0.051013,-1.161387,1.020473,Ethereum,1,0.000000e+00,0.006050


In [170]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="Class",
    width=800,
    height=500
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)

#### Table of Tradable Cryptocurrencies

In [171]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)


:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [172]:
# Print the total number of tradable cryptocurrencies
len(clustered_df.CoinName)


533